In [1]:
import tensorflow as tf

In [2]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [2]:
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
session = tf.Session(config=config)

In [3]:
session.list_devices()

[_DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 268435456),
 _DeviceAttributes(/job:localhost/replica:0/task:0/device:GPU:0, GPU, 3598673510)]

In [4]:
import keras

import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
%matplotlib inline

Using TensorFlow backend.


In [5]:
from keras.layers import Input, LSTM, RepeatVector, Dense, Embedding, Activation, Masking, SimpleRNN
from keras.layers.wrappers import TimeDistributed
from keras.models import Model, Sequential
from keras import backend as K

from keras.utils import to_categorical

In [6]:
import pickle

In [7]:
from sklearn.model_selection import train_test_split

### Constants&data

In [29]:
X = np.load("../data/DL_RNN_data_filtred.npy").astype(int)
Y = np.load("../data/DL_RNN_data_filtred_y.npy")
voc = pickle.load(open("../data/l_to_n", 'rb'))
#mask = np.zeros_like(X)

In [30]:
Y.shape

(358476,)

In [34]:
X = np.vstack((X, X[Y == 1]))

In [36]:
Y = np.hstack((Y, Y[Y == 1]))

In [37]:
train_X, test_X, train_y,  test_y = train_test_split(X, Y)

In [38]:
VOC_SIZE = len(voc)
emb_size = 50


#### Net

In [39]:
encoder_input = Input(shape=(None,))

#encoder_mask_layer = Masking(mask_value=3, input_shape=(t_s,))
#encoder_mask = encoder_mask_layer(encoder_input)

encoder_embedding_layer = Embedding(VOC_SIZE, emb_size, mask_zero=True)
encoder_embedding = encoder_embedding_layer(encoder_input)

encoder_recurent_layer_1 = LSTM(100,return_state=True, return_sequences=True )
encoder_recurent_1, encoder_h_1, encoder_c_1 = encoder_recurent_layer_1(encoder_embedding)

encoder_recurent_layer_2 = LSTM(100,return_state=True, return_sequences=False )
encoder_recurent_2, encoder_h_2, encoder_c_2 = encoder_recurent_layer_2(encoder_recurent_1)

encoder_dense_layer = Dense(300, activation='relu')
encoder_dense = encoder_dense_layer(encoder_h_2)

encoder_out_layer = Dense(2, activation='softmax')
encoder_out = encoder_out_layer(encoder_dense)

In [40]:
model = Model([encoder_input], encoder_out)

##### Learning

In [41]:
model.compile(optimizer='adam', loss='categorical_crossentropy')

In [42]:
learning_progress = model.fit(train_X, to_categorical(train_y), batch_size=100,
                                    epochs=1,
                                    validation_split=0.05)

Train on 347449 samples, validate on 18287 samples
Epoch 1/1
347449/347449 [==============================] - 667s 2ms/step - loss: 0.6650 - val_loss: 0.6584


In [43]:
pred_0 = model.predict(test_X)

In [44]:
pred_0

array([[ 0.26486185,  0.73513812],
       [ 0.48389885,  0.51610118],
       [ 0.47746864,  0.52253133],
       ..., 
       [ 0.45659173,  0.54340827],
       [ 0.30028293,  0.6997171 ],
       [ 0.46192107,  0.5380789 ]], dtype=float32)

In [45]:
test_y[:20]

array([1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0])

In [46]:
pred_0[:,1][:20]

array([ 0.73513812,  0.51610118,  0.52253133,  0.51026392,  0.53762585,
        0.73354   ,  0.6891371 ,  0.48269895,  0.68734521,  0.43411648,
        0.61103612,  0.27308515,  0.65886497,  0.62214124,  0.62973791,
        0.74868953,  0.51291007,  0.52440429,  0.76187366,  0.6770907 ], dtype=float32)

In [47]:
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score

In [48]:
roc_auc_score(test_y, pred_0[:,1] > 0.5) 

0.58703502471850344

In [49]:
f1_score(test_y, pred_0[:,1] > 0.5) 

0.69188091157239895

In [50]:
accuracy_score(test_y, pred_0[:,1] > 0.5)

0.60186035829122642

In [ ]:
mask = test_y == 1

In [ ]:
accuracy_score(test_y[mask], pred_0[mask][:,1] > 0.5)

In [1]:
print("saa")

saa
